# Load Modules and Data

In [1]:
import re
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt  
# import nltk
# from nltk.tokenize import sent_tokenize, word_tokenize
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
from tqdm import tqdm
# from nltk.corpus import stopwords
# stopwordEn = stopwords.words('english')
# from nltk.corpus import wordnet
import pickle
import multiprocessing as mp
import gensim
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore
from gensim.test.utils import datapath
import pyLDAvis.gensim
import time

print('CPU numbers:',mp.cpu_count())
def _apply_df(args):
    df, func, kwargs = args
    return df.apply(func, **kwargs)
def apply_by_multiprocessing(df, func, **kwargs):
#     print(kwargs)
    workers = kwargs.pop('workers')
    pool = mp.Pool(processes=workers)
    result = pool.map(_apply_df, [(d, func, kwargs) for d in np.array_split(df, workers)])
    pool.close()
    return pd.concat(list(result))
#apply_by_multiprocessing(fullset['Text'], processText, workers=cores)
def lemmaWord(word):
    lemma = wordnet.morphy(word)
    if lemma is not None:
        return lemma
    else:
        return word
def processText(text,lemma=False, gram=1):
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b|@\w+|#', '', text, flags=re.MULTILINE) #delete URL, #hashtag# , and @xxx
    tokens = word_tokenize(text)
    whitelist = ["n't", "not", "no"]
    tokens = [lemmaWord(i.lower()) if lemma else i.lower() for i in tokens if (i.lower() not in stopwordEn or i.lower() in whitelist) and i.isalpha()]
    if gram<=1:
        return tokens
    else:
        return [' '.join(i) for i in nltk.ngrams(tokens, gram)]
    
dates = range(23,30)

CPU numbers: 32


In [5]:
# dfs_news, dfs_tweets={},{}

# for d in tqdm(dates):
#     with open('./IEEE_news/df_03{}_news.pickle'.format(d), 'rb') as handle:
#         dfs_news[d] = pickle.load(handle)
#     with open('./IEEE_tweets/df_03{}_tweets.pickle'.format(d), 'rb') as handle:
#         dfs_tweets[d] = pickle.load(handle)

100%|██████████| 7/7 [00:25<00:00,  3.67s/it]


# Data Prepareation
Tweets: extract unique original and retweets

News: sentence level tokenization

In [2]:
daily_tweets = {}
def preDailyTweets(d):
    df_tweets_retweets = dfs_tweets[d][dfs_tweets[d]['re_full_text']!=''].iloc[:,[0,2]].drop_duplicates('re_full_text')
    df_tweets_retweets = df_tweets_retweets.rename(columns={'re_full_text':'full_text'})
    df_original_tweets = dfs_tweets[d][dfs_tweets[d]['re_full_text']==''].iloc[:,[0,1]].drop_duplicates('full_text')
    df_tweets = pd.concat([df_tweets_retweets,df_original_tweets]).reset_index()
    tweets_text = df_tweets.full_text
    tweets_token = apply_by_multiprocessing(tweets_text, processText, workers=32)
    day_tweets = pd.DataFrame({'text':tweets_text,'tokens':tweets_token})
    day_tweets.loc[:,'countTokens'] = [len(i) for i in day_tweets.tokens]
    day_tweets = day_tweets[day_tweets.countTokens!=0]#delete 0 tokens row
    return day_tweets

# daily_tweets[d] = preDailyTweets(d)

In [29]:
print(daily_tweets[d][daily_tweets[d].countTokens==71].tokens[239699])

['qanon', 'blast', 'past', 'gt', 'gt', 'b', 'f', 'p', 'gt', 'gt', 'q', 'gt', 'cue', 'gt', 'gt', 'clue', 'gt', 'gt', 'c', 'gt', 'gt', 'gt', 'gt', 'gt', 'gt', 'gt', 'c', 'b', 'k', 'gt', 'gt', 'supreme', 'court', 'brett', 'kavanaugh', 'gt', 'gt', 'likes', 'beer', 'gt', 'gt', 'corona', 'gt', 'gt', 'c', 'r', 'n', 'gt', 'gt', 'gt', 'gt', 'gt', 'gt', 'p', 'gt', 'gt', 'f', 'gt', 'gt', 'p', 'f', 'gt', 'gt', 'f', 'p', 'gt', 'gt', 'false', 'pandemic', 'gt', 'gt']


In [3]:
daily_news = {}
def preDailyNews(d):
    df_news = []
    for i in dfs_news[d].loc[:,'text']: # sentence level
        df_news.extend(sent_tokenize(i))
    news_text = pd.Series(df_news).unique()
    news_token = apply_by_multiprocessing(pd.Series(news_text), processText, workers=32)
    news_countTokens = [len(i) for i in news_token]
    day_news =  pd.DataFrame({'text':news_text,'tokens':news_token,'countTokens':news_countTokens})
    day_news = day_news[day_news.countTokens!=0]
    return day_news

# daily_news[d] = preDailyNews(d)

In [56]:
daily_news[d][daily_news[d].countTokens==161].text[26]

"The national animal - the tiger and the national bird - the peacock , go about their lives, unmindful of each other, at the Nagarhole National Park\nThe picture was clicked by reader Arvind ramakrishnan\nIf you have a picture and a story to share, send us the picture with your name and number at mybangaloremirror@timesgroup.com RELATED NEWS Happy Birthday Tiger Shroff: Disha Patani shares first dance video to... Mar 02, 2020\nDisha Patani celebrates Valentine's Day sans Tiger Shroff Feb 14, 2020\nWatch: Tiger Shroff aces a backflip in this new video Jan 16, 2020\nGALLERIES View more photos Vidhan Soudha gets a disinfection treatment Prabhu Ganesan spotted shooting in Mumbai India vs Australia 3rd ODI Most Popular Most Read Most commented Covid-19 in Bengaluru: ‘Curfew’ that br... Covid-19 in Bengaluru: ‘Curfew’ that broke the chain COVID-19 in Bengaluru: ‘Stock up on foo... COVID-19 in Bengaluru: ‘Stock up on food items for a week’ Karnataka intensifies lockdown; no publ... Karnataka 

In [66]:
print("Date:",d,' Tweets:',len(daily_tweets[d]),' News:',len(daily_news[d]))

Date: 23  Tweets: 390616  News: 1468669


In [6]:
# for d in tqdm(dates):
#     daily_tweets[d] = preDailyTweets(d)
#     del(dfs_tweets[d])
#     daily_news[d] = preDailyNews(d)
#     del(dfs_news[d])
#     print("Date:",d,' Tweets:',len(daily_tweets[d]),' News:',len(daily_news[d]))

 14%|█▍        | 1/7 [06:58<41:53, 418.96s/it]

Date: 23  Tweets: 390616  News: 1468669


 29%|██▊       | 2/7 [14:43<36:03, 432.73s/it]

Date: 24  Tweets: 418034  News: 1608189


 43%|████▎     | 3/7 [22:30<29:31, 442.78s/it]

...

Date: 27  Tweets: 602484  News: 1578489


 86%|████████▌ | 6/7 [43:36<06:55, 415.34s/it]

Date: 28  Tweets: 327211  News: 957997


100%|██████████| 7/7 [47:23<00:00, 406.26s/it]

Date: 29  Tweets: 123247  News: 616230


In [7]:
# Instead of constructing daily_xxx[date], seperate df into individual files
# with open('daily_tweets.pickle', 'wb') as handle:
#     pickle.dump(daily_tweets, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('daily_news.pickle', 'wb') as handle:
#     pickle.dump(daily_news, handle, protocol=pickle.HIGHEST_PROTOCOL)

# # seperate into mulitiple smaller files
# for d in dates:
#     with open(f'./data_combined/ge20_topic20-15-1g/senTo_Tweets_03{d}.pickle', 'wb') as handle:
#         pickle.dump(daily_tweets[d].iloc[:,1:], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [62]:
# Balance Sampling News
# num_tweets = df_topic_sent.groupby('day').apply(lambda x:np.sum([x[0],x[1],x[2]]))
for d in dates:
    daily_news[d] = daily_news[d].iloc[:,1:].sample(n=num_tweets[d], random_state=0)

In [63]:
daily_news[23]

,tokens,countTokens
240646,"[contrary, home, computers, allowed, work, fol...",12
514870,"[act, section, title, united, states, code, he...",10
650773,"[going, put, time, period, greenberg, called, ...",12
40098,"[even, talking, asian, heads, government, beca...",9
1123157,"[big, test, coming, nbn, vodafone, closed, num...",11
...,...,...
617404,"[veteran, village, kins, community, arizona, c...",22
894661,"[little, wonder, germany, announced, plans, us...",10
194548,"[agency, authority, crack, prosecute, illegal,...",8
683517,"[image, andrew, echo, sign, free, email, alert...",16


# Topic Mapping

In [15]:
ldamodel_all= LdaMulticore.load('./TopicModels/ldamodel_all.model')

In [22]:
###### DEPRECATED #######
# Use LDA model id2word attr to rebuild the dictionary
# dict_word2id = dict(ldamodel_all.id2word)
# dict_word2id = dict(zip(dict_word2id.values(), dict_word2id.keys()))

# def my_doc2bow(token):  # same as the corpus.dictionary.doc2bow()
#     token2id = []
#     for w in token:
#         if w in dict_word2id.keys():
#             token2id.append(dict_word2id[w])
#     if len(token2id)>0:
#         w_dict_count = pd.Series(token2id).value_counts().sort_index()
#         doc2bow = tuple(zip(w_dict_count.keys(),w_dict_count.values))
#         return doc2bow
#     else:
# #         print(token)
#         return []

# topic_mapping = [maxTopic(my_doc2bow(i)) for i in tqdm(tokens)] # use my_doc2bow instead of gensim.corora.dictionary

140570

In [18]:
def predictTopic(bow):
    if len(bow)>0:
        res = dict(ldamodel_all[bow]) # vectors of all topics
        return res
    
def maxTopic(res):
    if res is not None:
#         res2id = dict(zip(res.values(),res.keys()))
#         maxid = res2id[pd.Series(res).max()]
        return pd.Series(res).sort_values(ascending=False).index[0]
#         return maxid

In [16]:
#faster methods: use gensim.corpora.dictionary
#Directly use the subset dictionary that used in the previous training of LDA model 
with open('./TopicModels/dictionary_all.pickle', 'rb') as handle:
    subset_dictionary = pickle.load(handle)
from gensim import corpora
def mapTopics(tokens, df=False):
    topic_weight,topic_mapping = [],[]
    for i in tokens:
        bow = subset_dictionary.doc2bow(i)
        prob = predictTopic(bow)
        topic_weight.append(prob)
        topic_mapping.append(maxTopic(prob))
    if df:
        df_topics = pd.DataFrame({'tokens':tokens,'weightTopic':topic_weight,'maxTopic':topic_mapping})
        df_topics.dropna(inplace=True)
        return df_topics
    else:
        return topic_weight,topic_mapping

In [111]:
# Map topics to tweets
for d in tqdm(dates):
    topic_weight,topic_mapping = mapTopics(daily_tweets[d].tokens)
    daily_tweets[d].loc[:,'weightedTopic'] = topic_weight
    daily_tweets[d].loc[:,'maxTopic'] = topic_mapping    

100%|██████████| 7/7 [1:32:41<00:00, 794.43s/it]


In [327]:
# Map合并结果保存
# with open('daily_tweets.pickle', 'wb') as handle:
#     pickle.dump(daily_tweets, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
# Only tokens columns for tweets
# for d in range(23,30):
#     daily_tweets[d] = daily_tweets[d].loc[:,['tokens']]

# with open('daily_tweets_tokens.pickle', 'wb') as handle:
#     pickle.dump(daily_tweets, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [71]:
# Map topics to News
for d in tqdm(dates):
    topic_weight,topic_mapping = mapTopics(daily_news[d].tokens)
    daily_news[d].loc[:,'weightedTopic'] = topic_weight
    daily_news[d].loc[:,'maxTopic'] = topic_mapping    

100%|██████████| 7/7 [49:35<00:00, 425.06s/it]


In [91]:
# with open('daily_news_balanced.pickle', 'wb') as handle:
#     pickle.dump(daily_news, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Semantic Mapping

In [79]:
import tensorflow as tf
# tf.config.set_visible_devices([], 'GPU') #禁用GPU
with open('../SentimentAnalysis/tokenizer.pickle', 'rb') as handle:
    t = pickle.load(handle)

from tensorflow.keras.models import load_model
model_ge20 = load_model('../SentimentAnalysis/models/model_ge20')

In [80]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def text2Seq(tokens):
    text_seq = t.texts_to_sequences(tokens)
    return text_seq
def padSeq(seq):
    max_length = 21
    text_pad = pad_sequences(seq, maxlen=max_length, padding='post')
    return text_pad

In [88]:
# News
df_sent = {}
for d in tqdm(dates):
    text_seq = text2Seq(daily_news[d].tokens)
    text_pad = padSeq(text_seq)
    pred_sent_prob_ge20 = model_ge20.predict(text_pad)
    pred_sent_ge20 = [np.argmax(i) for i in pred_sent_prob_ge20]
    df_sent[d] = pd.DataFrame(model_ge20.predict(text_pad),columns=['s1','s2','s3'])
    df_sent[d].loc[:,'sent'] = pred_sent_ge20
    # combine df_sent into daily_news
    daily_news[d] = daily_news[d].reset_index(drop=True)
    daily_news[d] = pd.concat([daily_news[d],df_sent[d]], axis=1, sort=False)

100%|██████████| 7/7 [02:39<00:00, 22.74s/it]


In [47]:
#Tweets
# Import the results from Google colab, combine into daily_tweets
# with open('sent_tweets.pickle', 'rb') as handle:
#     sent_tweets = pickle.load(handle)

for d in dates:
    daily_tweets[d] = daily_tweets[d].reset_index(drop=True)
    daily_tweets[d] = pd.concat([daily_tweets[d],sent_tweets[d]], axis=1, sort=False)
# Final daily_tweets: combined all the mappings, tokens, and original text

# Semantic + Topics
Data aggregation

In [ ]:
with open('./data_combined/daily_tweets.pickle', 'rb') as handle:
    daily_tweets = pickle.load(handle)

In [52]:
daily_tweets[23][daily_tweets[23].sent==2].head()

,text,tokens,countTokens,weightedTopic,maxTopic,s1,s2,s3,sent
2,"@YSS_hardoi Indian Sages were, &amp; at presen...","[indian, sages, amp, present, great, yogi, amp...",26,{1: 0.9567716},1.0,0.044425,0.033047,0.922527,2
22,CORONA festival being celebrated in #Ahmedabad...,"[corona, festival, celebrated, ahmedabad, stup...",6,{14: 0.84159744},14.0,0.034268,0.315994,0.649738,2
26,'Go Corona' Masterstroke by Modiji today\n\nEn...,"[corona, masterstroke, modiji, today, entire, ...",29,"{1: 0.14526416, 8: 0.07074893, 12: 0.10902816,...",14.0,0.298927,0.055879,0.645194,2
51,Amongst all the corona virus chaos I am so hap...,"[amongst, corona, virus, chaos, happy, share, ...",26,"{0: 0.33327836, 3: 0.08759302, 16: 0.20877703,...",0.0,0.388928,0.063052,0.548020,2
75,We request @narendramodi ji must take the guid...,"[request, ji, must, take, guidance, great, sai...",14,{1: 0.9366297},1.0,0.026764,0.424973,0.548263,2


In [102]:
# for each topics count up sentiment labels
def getSentRes(dfs_daily):
    dfs_topic_sent = {}
    for d in tqdm(dates):
        topic_sent = dfs_daily[d].groupby('maxTopic').apply(lambda x: pd.Series(list(x['sent'])).value_counts())
        if isinstance(topic_sent, pd.Series): #23由于不明情况是df不是series
            df_topic_sent = pd.concat([pd.DataFrame(topic_sent[(i,)]).T for i in range(20) if (i,) in topic_sent.keys()],sort=False).reset_index(drop=True)
        else:
            df_topic_sent = topic_sent.reset_index(drop=True)
        #  df_topic_sent[[0,1,2]].max(axis=1)
        df_topic_sent.loc[:,'TopSent'] = [np.argmax([df_topic_sent[0][i],df_topic_sent[1][i],df_topic_sent[2][i]]) for i in range(len(df_topic_sent))]
        dfs_topic_sent[d] = df_topic_sent
        dfs_topic_sent[d].loc[:,'day']=d
    return dfs_topic_sent

dfs_topic_sent_tweets =  getSentRes(daily_tweets)
dfs_topic_sent_news = getSentRes(daily_news)

100%|██████████| 7/7 [00:02<00:00,  2.68it/s]


In [2]:
with open('./data_combined/dfs_topic_sent.pickle', 'rb') as handle:
    dfs_topic_sent = pickle.load(handle)
with open('./data_combined/dfs_topic_sent_news.pickle', 'rb') as handle:
    dfs_topic_sent_news = pickle.load(handle)
def aggSentRes(dfs_topic_sent):
    df = pd.concat([dfs_topic_sent[d] for d in dates]).reset_index()
    df = df.rename(columns={'index':'TopicID'})
    df.TopicID = df.TopicID + 1
    df.loc[:,'total'] = df[0]+df[1]+df[2]
    df.loc[:,'TopicScore'] = (df[2]-df[0])/df['total']
    return df.drop(['TopSent'],axis=1)
df_topic_sent = aggSentRes(dfs_topic_sent)
df_topic_sent_news = aggSentRes(dfs_topic_sent_news)

In [22]:
# with open('./data_combined/df_topic_sent.pickle', 'wb') as handle:
#     pickle.dump(df_topic_sent, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('./data_combined/df_topic_sent_news.pickle', 'wb') as handle:
#     pickle.dump(df_topic_sent_news, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
# [np.argmax([df_topic_sent[0][i],df_topic_sent[1][i],df_topic_sent[2][i]]) for i in range(len(df_topic_sent))]
df_topic_sent_news

,TopicID,0,1,2,day,total,TopicScore
0,1,1309,977,304,23,2590,-0.388031
1,2,5966,1637,649,23,8252,-0.644329
2,3,6659,1431,783,23,8873,-0.662234
3,4,50795,13653,659,23,65107,-0.770055
4,5,4789,1852,204,23,6845,-0.669832
...,...,...,...,...,...,...,...
135,16,7343,1391,557,29,9291,-0.730384
136,17,320,336,110,29,766,-0.274151
137,18,458,281,59,29,798,-0.500000
138,19,389,545,31,29,965,-0.370984
